# Basic simulation of electrodes in ESPResSo part II:
# Electrolyte capacitor and Poisson--Boltzmann theory

In this tutorial we are going to investigate [...] using **ESPResSo**. 

To work with this tutorial, you should be familiar with the following topics:

1. Setting up and running simulations in ESPResSo - creating particles,
   incorporating interactions.
   If you are unfamiliar with this, you can go through the respective tutorial
   in the `lennard_jones` folder.
2. Basic knowledge of classical electrostatics:
   Dipoles, surface and image charges
3. Reduced units, as described in the ESPResSo user guide
   https://espressomd.github.io/doc/introduction.html#on-units

## Introduction

Understanding the electric double layer (EDL) is crucial for the study
of a variety of systems, including colloidal suspensions, charged biological
macromolecules and membranes. Poisson-Boltzmann (PB) theory based on a mean-field formalism properly describes the behavior of Coulomb fluids composed of monovalent ions at low concentrations in the
vicinity of weakly charged interfaces. However, for strongly charged systems, where correlation and finite size effects begin to dominate the system dynamics, the PB theory falls inadequate. Our goal in this module is to demonstrate how coarse grained implicit solvent simualtions can corroborate some of these approximate theories.

Here, we look at the ion density profile between two dielectric walls with constant potential applied to them. 
In this case the inclusion of dielectric inhomogeneities demands a detailed calculation of the image
effects that involve the full solution of the Poisson equation on the fly. This is dealt in a computational cost effective way using ELCIC method, to treat the image charge effect in the presence of 2D dielectric bounding
interfaces.  

[...]

## Theoretical Background 

[...]

## 1. System setup 

First we import all ESPResSo features and external modules

In [ ]:
import espressomd
import numpy as np
import espressomd.electrostatics
import espressomd.electrostatic_extensions
from espressomd.interactions import *
import espressomd.visualization
import espressomd.observables
import espressomd.accumulators
from espressomd import shapes
import matplotlib.pyplot as plt
from scipy.special import *
from scipy import constants as const
from tqdm import tqdm


We define system dimensions and some physical parameters related to length, time
and energy scales of our system.
All physical parameters are defined in reduced units of length ($\sigma=1$;
Particle size), mass ($m=1$; Particle mass), time ($t=0.01 \tau$) and
elementary charge ($e=1$).

Another important length scale is the Bjerrum Length, which is the length at
which the electrostatic energy between two elementary charges is comparable to
the thermal energy $k_\mathrm{B}T$.
It is defined as $l_\mathrm{B}=\frac{1}{4\pi\epsilon_0\epsilon_rk_BT}$. 
In our case if we choose the ion size ($\sigma$) in simulations equivalent to a
typical experimental value for mono-atomar salt, 0.3 nm in real units, then the
Bjerrum length of water at room temperature, $\ell_\mathrm{B}=0.71 \,\mathrm{nm}$ is
$\ell_\mathrm{B}\sim 2$ in simulations units.

In [ ]:
# Keep lb=2 (water) and LJ units, convert to physical units using ION_SIZE

BJERRUM_LENGTH = 0.71        # Electrostatic prefactor passed to P3M ; prefactor=lB KBT/e2                

#Lennard-Jones  Parameters
LJ_SIGMA = 0.3              # Particle size nanometers, not point-like
LJ_EPSILON = 1.0
HS_ION_SIZE = 2**(1/6) * LJ_SIGMA

CONCENTRATION = 1e-3 # desired concentration 10 mmol/l
DISTANCE = 10 # 10 Debye lengths
N_IONPAIRS = 500

POTENTIAL_DIFF = 5.0

# Elementary charge 
q = np.array([1.0])  
types = {"Cation": 0, "Anion": 1  ,"Electrodes": 2}
charges = {"Cation": q[0], "Anion": -q[0]  }

In [ ]:
def get_box_dimension(concentration, distance, n_ionpairs=N_IONPAIRS):
    """ For a given number of particles, determine the lateral area of the box
    to match the desired concentration """

    # concentration is in mol/l, convert to 1/sigma**3
    rho = concentration * (const.Avogadro / const.liter) * const.nano**3
    debye_length = (4 * np.pi * BJERRUM_LENGTH * rho*2)**(-1./2) # desired Debye length in nm
    distance_in_nanometers = distance * debye_length
    
    box_volume = n_ionpairs / rho
    area = box_volume / (distance_in_nanometers - 2*HS_ION_SIZE) # account for finite ion size in density calculation
    l_xy = np.sqrt(area)

    return rho,l_xy, debye_length
    # determine the box size to achieve the desired density
    #return (rho * distance**2)**(1.0/3.0)

rho, box_l_xy, DEBYE_LENGTH = get_box_dimension(CONCENTRATION,DISTANCE,N_IONPAIRS)
box_l_z = DISTANCE * DEBYE_LENGTH

In [ ]:
ELC_GAP = 6*box_l_z
system = espressomd.System(box_l=[box_l_xy, box_l_xy, box_l_z+ELC_GAP])
system.time_step = 0.01

We now set up an electrolyte solution made of monovalent cations (Np) and anions
(Np) between two metallic electrodes at constant potential. 
Firstly, we add two wall constraints at $z=0$ and $z=L_z$ to stop particles from
crossing the boundaries.
Refer to  [espressomd.constraints.ShapeBasedConstraint]
(https://espressomd.github.io/doc/espressomd.html#espressomd.constraints.ShapeBasedConstraint)
in the documentation to set up constraints. Then we place all the ion pairs at
random positions between the electrodes.  

In [ ]:
# ADDING ELECTRODES
wall_offset=0

# (BOTTOM WALL) normal pointing in the +z direction, at z=wall_offset 
floor = espressomd.shapes.Wall(normal=[0, 0, 1], dist=wall_offset)
c1 = system.constraints.add(
    particle_type=types["Electrodes"], penetrable=False, shape=floor)

# (TOP WALL) normal pointing in the -z direction, at z=box_l_z-wall_offset
ceil = espressomd.shapes.Wall(normal=[0, 0, -1], dist=-(box_l_z - wall_offset))   
c2 = system.constraints.add(
    particle_type=types["Electrodes"], penetrable=False, shape=ceil)


# ADDING IONPAIRS 
offset=10 # To stop the particle to cross the wall at integration 
Init_part_btw_z1=0+offset 
Init_part_btw_z2=box_l_z-offset
ion_pos=np.empty((3),dtype=float)

for i in range (N_IONPAIRS):
    ion_pos[0] = np.random.random(1) * system.box_l[0]
    ion_pos[1] = np.random.random(1) * system.box_l[1]
    ion_pos[2] = np.random.random(1) * (Init_part_btw_z2-Init_part_btw_z1) + Init_part_btw_z1
    system.part.add(pos=ion_pos, type=types["Cation"]  , q=charges["Cation"])
    
for i in range (N_IONPAIRS):
    ion_pos[0] = np.random.random(1) * system.box_l[0]
    ion_pos[1] = np.random.random(1) * system.box_l[1]
    ion_pos[2] = np.random.random(1) * (Init_part_btw_z2-Init_part_btw_z1) + Init_part_btw_z1
    system.part.add(pos=ion_pos, type=types["Anion"]  , q=charges["Anion"])

Now, we add interactions:
For excluded volume interactions, we add WCA kind of potential. 
For the (2D+h) electrostatic with dielectrics we choose the ELCIC with P3M.

Refer to the documentation to set up the
[WCA interaction](https://espressomd.github.io/doc/espressomd.html#espressomd.interactions.WCAInteraction) 
under [Non-bonded](https://espressomd.github.io/doc/inter_non-bonded.html)
section.

Refer the documentation to set up
[ELCIC with P3M](https://espressomd.github.io/doc/electrostatics.html#electrostatic-layer-correction-elc)
under the [electrostatics](https://espressomd.github.io/doc/electrostatics.html)
section. 

In [ ]:
#***************************************************************
#                   Adding WCA interactions 
#***************************************************************

for  key, val in types.items():
    for key1, val1 in types.items():
        system.non_bonded_inter[val, val1].wca.set_params(epsilon=LJ_EPSILON, sigma=LJ_SIGMA)

#***************************************************************
#                 Adding ELC-P3M electrostatic solver   
#***************************************************************

delta_mid_top = -1.0   #(Fully metallic case both -1)                 
delta_mid_bot = -1.0

accuracy = 1e-3
check_accuracy = 1e-3
p3m = espressomd.electrostatics.P3M(prefactor=BJERRUM_LENGTH,
                                    accuracy=accuracy, 
                                    #mesh=[58, 58, 70], 
                                    #mesh=[128, 128, 64], 
                                    #cao=6,
                                    verbose=True,
                                    tune=True,
                                    timings=1,
                                    )
#system.electrostatics.solver = p3m

elc = espressomd.electrostatics.ELC(actor=p3m,
                                    gap_size=ELC_GAP,
                                    const_pot=True,
                                    pot_diff=POTENTIAL_DIFF,
                                    maxPWerror=check_accuracy,
                                    delta_mid_bot=delta_mid_bot,
                                    delta_mid_top=delta_mid_top)
system.electrostatics.solver = elc

## 2. Equilibration

Before we can start the simulation, we need to remove the overlap between particles to avoid large forces which would crash the simulation. For this, we use the steepest descent integrator with a relative convergence criterion for forces and energies.



In [ ]:
#  Relax the overlaps with steepest descent

system.integrator.set_steepest_descent(f_max=10, gamma=50.0,
                                       max_displacement=0.02)
system.integrator.run(1000)
system.integrator.set_vv() # Switch bach to velocity Verlet 

# Add thermostat 
thermostat_seed = np.random.randint(np.random.randint(1000000))
system.thermostat.set_langevin(kT=1.0, gamma=0.1, seed=thermostat_seed)

Convergence after $t\sim25$ time units, possible to run up to $t=100$ here...
This is a total of 10.000 time steps.

In [ ]:
# Integration parameters
STEPS_PER_SAMPLE = 200
N_SAMPLES = 25
N_PART = 2* N_IONPAIRS

times = np.zeros(N_SAMPLES)
e_total = np.zeros_like(times)

In [ ]:
for i in tqdm(range(N_SAMPLES)):
    times[i] = system.time
    energy = system.analysis.energy()
    e_total[i] = energy['total']
    system.integrator.run(STEPS_PER_SAMPLE)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(times, e_total)
plt.xlabel('t')
plt.ylabel('E')
plt.show()

# 3. Calculate and analyze ion profile

In [ ]:
#*************************************************************
#  PART 2:       SETTING ACCUMULATORS FOR DENSITY PROFILE 
#**************************************************************
# PRODUCTION RUNS

N_SAMPLES = 100

Ion_id=[]
Cations = system.part.select(type=types["Cation"])
Cations_id=[]
for i in Cations:
    Cations_id.append(i.id)
    Ion_id.append(i.id)
    
Anions = system.part.select(type=types["Anion"])
Anions_id=[]
for i in Anions:
    Anions_id.append(i.id)
    Ion_id.append(i.id)

bin_width = DEBYE_LENGTH/25
n_z_bins = int((system.box_l[2] - ELC_GAP) / bin_width)
    

  
# Accumulator 1 : observable::Density_Profile
density_profile_cation = espressomd.observables.DensityProfile(ids=Cations_id,
                                                       n_x_bins=1,
                                                       n_y_bins=1,
                                                       n_z_bins=n_z_bins,
                                                       min_x=0,
                                                       min_y=0,
                                                       min_z=0,
                                                       max_x=system.box_l[0],
                                                       max_y=system.box_l[1],
                                                       max_z=system.box_l[2] - ELC_GAP)

density_accumulator_cation = espressomd.accumulators.MeanVarianceCalculator(obs=density_profile_cation, delta_N=20)


density_profile_anion = espressomd.observables.DensityProfile(ids=Anions_id,
                                                       n_x_bins=1,
                                                       n_y_bins=1,
                                                       n_z_bins=n_z_bins,
                                                       min_x=0,
                                                       min_y=0,
                                                       min_z=0,
                                                       max_x=system.box_l[0],
                                                       max_y=system.box_l[1],
                                                       max_z=system.box_l[2] - ELC_GAP)

density_accumulator_anion = espressomd.accumulators.MeanVarianceCalculator(obs=density_profile_anion, delta_N=20)


system.auto_update_accumulators.clear()
system.auto_update_accumulators.add(density_accumulator_cation)
system.auto_update_accumulators.add(density_accumulator_anion)
    
times=[]
e_total=[]
for tm in tqdm(range(N_SAMPLES)):
    system.integrator.run(STEPS_PER_SAMPLE)
    times.append( system.time)
    energy = system.analysis.energy()
    e_total.append( energy['total'])   

In [ ]:
zs = density_profile_anion.bin_centers()[0, 0, :, 2]
cation_profile_mean = density_accumulator_cation.mean()[0, 0, :]
anion_profile_mean = density_accumulator_anion.mean()[0, 0, :]

In [ ]:
def gouy_chapman_potential(x, debye_length, phi_0):
    kappa = 1./debye_length
    return 2*np.log((1 + np.tanh(1./4*(phi_0) * np.exp(-kappa*x))) \
                  / (1 - np.tanh(1./4*(phi_0) * np.exp(-kappa*x))))

def gouy_chapman_density(x, c0, debye_length, phi_0):
    phi = gouy_chapman_potential(x, debye_length, phi_0)
    return c0/2. * np.exp(-phi)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(zs, cation_profile_mean, color='b', label='cation')
plt.plot(zs, anion_profile_mean, color='r', label='anion')
plt.plot(zs, cation_profile_mean + anion_profile_mean, color='k', label='total')

x = np.linspace(HS_ION_SIZE, box_l_z-HS_ION_SIZE, 100)
plt.plot(x, (gouy_chapman_density(x, CONCENTRATION, DEBYE_LENGTH,-POTENTIAL_DIFF/2.) \
         + gouy_chapman_density(box_l_z-HS_ION_SIZE-x, CONCENTRATION, DEBYE_LENGTH,POTENTIAL_DIFF/2.))/2., color='r', ls='--')
plt.plot(x, (gouy_chapman_density(box_l_z-HS_ION_SIZE-x, CONCENTRATION, DEBYE_LENGTH,-POTENTIAL_DIFF/2.) \
         + gouy_chapman_density(x, CONCENTRATION, DEBYE_LENGTH,POTENTIAL_DIFF/2.) )/2., color='b', ls='--')
plt.plot(x, (gouy_chapman_density(x, CONCENTRATION, DEBYE_LENGTH,-POTENTIAL_DIFF/2.) \
         + gouy_chapman_density(box_l_z-HS_ION_SIZE-x, CONCENTRATION, DEBYE_LENGTH,POTENTIAL_DIFF/2.))/2 \
         + (gouy_chapman_density(box_l_z-HS_ION_SIZE-x, CONCENTRATION, DEBYE_LENGTH,-POTENTIAL_DIFF/2.) \
         + gouy_chapman_density(x, CONCENTRATION, DEBYE_LENGTH,POTENTIAL_DIFF/2.) )/2., color='k', ls='--', lw=2)

plt.legend()
plt.xlabel(r'$z\,\mathrm{[nm]}$')
plt.ylabel(r'$\rho(z)\,\mathrm{[mol/l]}$')

Graham equation test

In [ ]:
#Test sigma with Graham equation
sigma_left = np.sum((cation_profile_mean-anion_profile_mean)[:int(len(zs)/2.)]) * (zs[1]-zs[0])
sigma_right = np.sum((+cation_profile_mean-anion_profile_mean)[int(len(zs)/2.):]) * (zs[1]-zs[0])
sigma_graham = np.sinh(POTENTIAL_DIFF/4.) * np.sqrt(2*rho/(np.pi*BJERRUM_LENGTH))
# sigma in e/nm^2
sigma_left, sigma_right, sigma_graham


From here needs to be fixed for units...

In [ ]:
# TODO: fix units
f, ax = plt.subplots(figsize=(10, 6))

dz = (zs[1]-zs[0])*const.nano
chargedensity = (cation_profile_mean-anion_profile_mean)*const.elementary_charge/const.nano**3 
E_ind_SI = 1./const.epsilon_0 * np.cumsum(chargedensity*dz)
# integration constant: zero field in the center
E_ind_SI -= E_ind_SI.min()
E_ind = E_ind_SI / (const.elementary_charge / (const.Boltzmann * 300) / const.nano)
ax2 = plt.twinx()

ax.plot(zs,E_ind_SI)
ax2.plot(zs,E_ind)
ax.set_xlabel(r'$z\,\mathrm{[nm]}$')
ax.set_ylabel(r'$E_\mathrm{ind}\,\mathrm{[V/m]}$')
ax2.set_ylabel(r'$E_\mathrm{ind}\,\mathrm{[(k_\mathrm{B}T/e)/nm]}$')

In ELC-IC, the applied voltage between the electrodes consists of two
contributions,
$$ \Delta \phi_\mathrm{applied} = \Delta \phi_\mathrm{ind} + \phi_\mathrm{bat},$$
where $\Delta \phi_\mathrm{ind}$ denotes the potential drop due to the
polarization between the parallel electrodes (that is exactly counterbalanced by
the induced image charges).
In the constant potential ensemble, we obtain the target potential 
$\Delta \phi_\mathrm{bat}$ by superposing a homogeneous electric field created by
the induced charges with the corresponding battery voltage.
The contribution $\Delta \phi_\mathrm{ind}$ is calculated on the fly basically
for since the dipole moment $P_z$ is calculated in ELC anyway.
Let us explicitely calculate these two contributions (in reduced units):

In [ ]:
# TODO: fix units
dz = zs[1]-zs[0]
phi_ind = -2*np.sum(E_ind[:int(len(E_ind)/2)]) * dz
phi_el  = (4*np.pi*BJERRUM_LENGTH) * box_l_z * sigma_left
phi_tot = phi_ind + phi_el
print(phi_el, phi_ind, phi_tot)

Potential drop at one electrode:

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
phi_ind_z = -2*np.cumsum(E_ind[:int(len(E_ind)/2)]*(zs[1]-zs[0]))
phi_el_z = zs[:int(len(zs)/2.)] * sigma_left * const.elementary_charge / const.nano / const.epsilon_0 / (const.elementary_charge / (const.Boltzmann * 300))
plt.plot(zs[:],np.hstack([phi_ind_z, -phi_ind_z[::-1]+2*phi_ind_z[-1]]), label=r'$\phi_\mathrm{ind}$')
plt.plot(zs[:],np.hstack([phi_el_z, -phi_el_z[::-1]+2*phi_el_z[-1]]), label=r'$\phi_\mathrm{bat}$')
plt.plot(zs[:],np.hstack([phi_el_z, -phi_el_z[::-1]+2*phi_el_z[-1]]) + np.hstack([phi_ind_z, -phi_ind_z[::-1]+2*phi_ind_z[-1]]), label=r'$\phi_\mathrm{applied}$')
plt.legend()
plt.xlabel(r'$z\,\mathrm{[nm]}$')
plt.ylabel(r'$\phi\,[k_\mathrm{B}T/e]$')

## 4. Differential capacitance

In [ ]:
# todo

## References

<a id='[1]'></a>[1] 
